In [8]:
from dash_website import DIMENSIONS, ALL_CATEGORIES
import pandas as pd 

phenotypic_table = pd.read_csv("../data/page4_correlations/ResidualsCorrelations/ResidualsCorrelations_bestmodels___Age_test.csv", index_col=0)

In [9]:
list_indexes = []
for dimension_1 in DIMENSIONS:
    for dimension_2 in DIMENSIONS:
        list_indexes.append([dimension_1, dimension_2])
indexes = pd.MultiIndex.from_tuples(list_indexes, names=["dimension_1", "dimension_2"])
phenotypic = pd.DataFrame(None, index=indexes, columns=["correlation"])

In [10]:
phenotypic_raw = phenotypic_table.stack(dropna=False).rename(index={"*": "set", "*instances01": "set_instances01", "*instances1.5x": "set_instances1.5x", "*instances23": "set_instances23"})

In [11]:
phenotypic_raw.index.names = ["dimension_1", "dimension_2"]
phenotypic_raw.name = "correlation"
phenotypic_raw

dimension_1       dimension_2            
set               set                        1.000000
                  set_instances01            0.838413
                  set_instances1.5x          0.998501
                  set_instances23            0.774242
                  Abdomen                    0.375136
                                               ...   
PhysicalActivity  MusculoskeletalHips        0.137794
                  MusculoskeletalKnees       0.110722
                  MusculoskeletalFullBody    0.159431
                  MusculoskeletalScalars     0.125744
                  PhysicalActivity           1.000000
Name: correlation, Length: 1089, dtype: float64

In [12]:
phenotypic["correlation"] = phenotypic_raw